In [37]:
import torch
from utils import LB_utils_special
from scipy.special import gammaln, xlogy
import numpy as np

## compute KL divergence from samples

we draw $x_i$ from a Gaussian, transform it by a softmax and compute $\sum_i \log p(x_i) / \log q(x_i)$ where $p$ is the Dirichlet and $q$ is 

OK maybe just use MMD? this stuff doesn't seem to work

In [38]:
def mv_norm_pdf(X, mu, Sigma):
    return(torch.exp(torch.distributions.multivariate_normal.MultivariateNormal(mu, Sigma).log_prob(X)))

def LB_alpha(mu, Sigma):
    return(LB_utils_special.get_alpha_from_Normal(mu, Sigma))

def dir_sm_pdf(x, alpha):
    
    #softmax X row-wise
    x_sm = torch.exp(x) / torch.exp(x).sum()
    coef = torch.sum(torch.special.gammaln(alpha)) - torch.special.gammaln(torch.sum(alpha))
    r = torch.exp(-coef + torch.sum((torch.xlogy(alpha, x_sm.T)).T, 0))
    return(r)

"""
def softmax_transform(x):
    
    exp_ = np.exp(x)
    return(exp_/exp_.sum())

def dirichlet_pdf(x, alpha):
    
    alpha = np.array(alpha)
    x = np.array(x)
    coef = np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))  

    return(np.exp(- coef+ np.sum((xlogy(alpha-1, x.T)).T, 0)))

def dirichlet_t_softmax_pdf(x, alpha):
    
    alpha = np.array(alpha)
    x_sm = softmax_transform(np.array(x))
    coef = np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))
    
    return(np.exp(- coef+ np.sum((xlogy(alpha, x_sm.T)).T, 0)))
"""

'\ndef softmax_transform(x):\n    \n    exp_ = np.exp(x)\n    return(exp_/exp_.sum())\n\ndef dirichlet_pdf(x, alpha):\n    \n    alpha = np.array(alpha)\n    x = np.array(x)\n    coef = np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))  \n\n    return(np.exp(- coef+ np.sum((xlogy(alpha-1, x.T)).T, 0)))\n\ndef dirichlet_t_softmax_pdf(x, alpha):\n    \n    alpha = np.array(alpha)\n    x_sm = softmax_transform(np.array(x))\n    coef = np.sum(gammaln(alpha)) - gammaln(np.sum(alpha))\n    \n    return(np.exp(- coef+ np.sum((xlogy(alpha, x_sm.T)).T, 0)))\n'

In [39]:
test_mu = torch.tensor([-3, 1, 2])
test_Sigma = torch.eye(3)

test_alpha = LB_alpha(test_mu.reshape(1,-1), test_Sigma.reshape(1,3,3))
print(test_alpha)

tensor([[ 0.4472,  6.5518, 17.2368]])


In [40]:
test_x = torch.tensor([-2,1,1])

# test numpy implementation
print(dirichlet_t_softmax_pdf(test_x.numpy(), test_alpha.numpy()))
print(dir_sm_pdf(test_x, test_alpha))
print(mv_norm_pdf(test_x.float(), test_mu, test_Sigma))

[0.01559771]
tensor([0.0156])
tensor(0.0234)


In [70]:
# draw 100 samples and compute the KL divergence

m = torch.distributions.multivariate_normal.MultivariateNormal(test_mu.float(), test_Sigma)
X100 = m.sample(torch.Size([1000]))
print(X100.size())
print(X100[:10])

p_dir = torch.tensor([dir_sm_pdf(x_, test_alpha) for x_ in X100])
p_norm = mv_norm_pdf(X100, test_mu, test_Sigma)
KL_div = torch.mean(torch.log(p_norm) - torch.log(p_dir))
print(KL_div)

torch.Size([1000, 3])
tensor([[-4.1200,  2.1378,  2.4876],
        [-2.5344,  1.1133,  3.2828],
        [-1.2609,  1.5332,  2.3116],
        [-1.2363,  0.4464,  3.1137],
        [-2.3732,  0.7320, -0.0565],
        [-2.9778,  0.6861,  2.4239],
        [-3.3694,  1.9081,  1.1439],
        [-2.4071, -0.4569,  3.1743],
        [-5.0944, -0.6284,  1.3238],
        [-5.0899,  1.8738,  1.5847]])
tensor(2.1817)


In [72]:
# draw 100 samples and compute the KL divergence

m = torch.distributions.multivariate_normal.MultivariateNormal(test_mu.float(), test_Sigma)
X100_2 = m.sample(torch.Size([1000]))
print(X100_2.size())
print(X100_2[:10])

p_norm_2 = mv_norm_pdf(X100_2, test_mu, test_Sigma)
p_norm = mv_norm_pdf(X100, test_mu, test_Sigma)
KL_div = torch.mean(torch.log(p_norm_2) - torch.log(p_norm))
print(KL_div)

torch.Size([1000, 3])
tensor([[-3.2722,  0.3552,  2.5232],
        [-1.2532,  2.5559,  1.7372],
        [-2.0092,  1.7802,  2.9392],
        [-2.0377,  2.6029,  1.8011],
        [-3.5180,  1.4854,  1.9742],
        [-3.8944,  0.0497,  1.6086],
        [-3.8251,  1.0022,  2.5863],
        [-2.6579,  0.6804,  1.0628],
        [-1.7159,  1.3588,  1.3958],
        [-4.1258,  0.2075,  2.1199]])
tensor(0.0090)


In [87]:
# compute with uniform simplex sampling

simplex_samples = torch.distributions.dirichlet.Dirichlet(torch.tensor([1., 1., 1.])).sample(torch.Size([1000]))

# translate to R
simplex_samples_R = torch.log(simplex_samples)

eps = 1e-20
p_norm = mv_norm_pdf(simplex_samples_R, test_mu, test_Sigma) + eps
p_dir = torch.tensor([dir_sm_pdf(x_, test_alpha) for x_ in simplex_samples_R]) + eps
KL_div = torch.mean(p_norm * (torch.log(p_norm) - torch.log(p_dir)))
print(KL_div)

tensor(-8.9202e-05)


In [88]:
print(p_norm)
print(p_dir)

tensor([1.1044e-05, 2.0553e-08, 1.0035e-06, 1.4146e-04, 2.0724e-14, 1.9574e-06,
        4.9713e-20, 1.2729e-04, 2.9760e-08, 2.3882e-07, 3.4909e-16, 1.5826e-08,
        4.8508e-08, 1.4408e-07, 2.4734e-04, 1.9952e-05, 1.3619e-06, 1.6097e-04,
        7.5394e-07, 8.3826e-09, 7.5371e-05, 2.7776e-06, 4.6535e-09, 1.3043e-04,
        4.8727e-09, 7.0109e-06, 7.7268e-07, 1.8521e-04, 8.6855e-14, 3.6014e-05,
        5.6971e-08, 2.0311e-05, 1.1687e-05, 2.7786e-06, 3.3492e-06, 1.8003e-04,
        3.2272e-06, 1.7376e-05, 2.8358e-07, 9.1528e-06, 3.0678e-08, 1.5050e-08,
        4.1353e-08, 2.2774e-08, 3.7978e-05, 2.9261e-07, 7.3712e-09, 1.6506e-04,
        4.5745e-09, 2.0429e-06, 1.5122e-09, 1.1353e-05, 1.4450e-05, 9.2232e-18,
        2.1464e-08, 1.0862e-08, 1.5152e-09, 7.5193e-09, 2.5203e-07, 2.6184e-09,
        1.3386e-04, 2.7887e-05, 4.4742e-06, 2.1554e-11, 3.8686e-10, 2.4373e-07,
        8.4671e-05, 2.0845e-04, 1.7529e-06, 1.4554e-11, 2.2699e-08, 4.7987e-08,
        6.9041e-05, 1.0550e-08, 1.1917e-